In [123]:
#in anaconda commandprompt:
    #conda install pytorch torchvision cpuonly -c pytorch
    
    
    #pip install opencv
    
    
    #pip install jupyterlab ultralytics
    # or
    #pip install ultralytics
    or
    #pip install ultralytics==8.2.76


# you can train your own model in yolo and use it as well

# Base detection and tracker

In [36]:
#Video_Path = r'./videos/video360p.mp4'
from ultralytics import YOLO
import cv2
from IPython.display import clear_output

model= YOLO('yolov8n.pt')
Video_Path = r"C:\Users\KeyhanAzarjoo\Python Training Code\Python-Yolo8-object-Detection\videos\video1111.mp4"

In [38]:
def basedetectionandtracker(Path):
    cap = cv2.VideoCapture(Path)
    ret = True
    while ret:
        ret, frame = cap.read()
        clear_output(wait=True)
        if ret:
            
            # detect objects
            # track object
            results = model.track(frame, persist=True)
        
            # plot results    
            frame_ = results[0].plot()
        
            # visualize
            cv2.imshow('frame', frame_)
            if cv2.waitKey(1) & 0xff == ord('q'):
                break

In [14]:
basedetectionandtracker(Video_Path)


0: 384x640 10 persons, 47.5ms
Speed: 2.0ms preprocess, 47.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


# view webcam 

In [1]:
import cv2

In [3]:
cap = cv2.VideoCapture(0) # 0 is the webcam
ret = True

In [ ]:
while ret:
    ret, frame = cap.read()
    if ret:

        cv2.imshow('frame', frame)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break

# Detect and Track any object in YOLO8 from file and webcam

In [1]:
import cv2
#import torch
from ultralytics import YOLO
from IPython.display import clear_output

# Function to process video with optimizations
def process_video(video_path, frame_skip=1, ConfidenceThreshold = 0.75, ShowVideo = False, Resolution = (0, 0)):
    # Load video
    model= YOLO('yolov8n.pt')
    cap = cv2.VideoCapture(video_path)


    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        clear_output(wait=True)
        if not ret:
            break
         # Skip frames if needed
        if frame_count % frame_skip != 0:
             frame_count += 1
             continue

        # Optionally resize the frame for faster processing
        if Resolution != (0,0):
            frame = cv2.resize(frame, Resolution)   

        
        # Use the model to track objects in the frame
        results = model.track(frame, show=False, persist=True, conf=ConfidenceThreshold)
        if ShowVideo:
            annotated_frame = results[0].plot()
            cv2.imshow("FRAME",annotated_frame)
            del annotated_frame

        del frame 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()
# Example usage
# Load a lighter YOLO model (e.g., YOLOv8-tiny) and set the video path



In [ ]:
# from file

In [3]:
Video_Path = r"C:\Users\KeyhanAzarjoo\Python Training Code\Python-Yolo8-object-Detection\test.MP4"
test = process_video(Video_Path, frame_skip=2, ConfidenceThreshold = 0.25, ShowVideo = True, Resolution = (0, 0))

In [ ]:
# from webcam

In [ ]:
Video_Path = 0 # it take a bit time
test = process_video(Video_Path, frame_skip=2, ConfidenceThreshold = 0.25, ShowVideo = True, Resolution = (0, 0))
#test = process_video(Video_Path, frame_skip=2, ConfidenceThreshold = 0.25, ShowVideo = True, Resolution = (1080, 720)) # to chenge the resolution

# Detect and Track any object in YOLO8 from stream Url Like Youtube

In [84]:
import cv2
from ultralytics import YOLO
from vidgear.gears import CamGear   
from IPython.display import clear_output


# Function to process video with optimizations
def process_live_video(video_url, frame_skip=1, ConfidenceThreshold = 0.75, ShowVideo = False, Resolution = (0, 0)):
    stream = CamGear(source=video_url, stream_mode = True, logging=False).start()
    # Load video
    model= YOLO('yolov8n.pt')

    frame_count = 0
    while True:
        frame = stream.read()
        clear_output(wait=True)
        if frame is None:
            break
        clear_output(wait=True)

        if frame_count % frame_skip != 0:
             frame_count += 1
             continue
            
        # Optionally resize the frame for faster processing
        if Resolution != (0,0):
            frame = cv2.resize(frame, Resolution)   

        results = model.track(frame, show=False, persist=True, conf=ConfidenceThreshold)
        if ShowVideo:
            annotated_frame = results[0].plot()
            cv2.imshow("FRAME",annotated_frame)
            del annotated_frame

        del frame 
        if cv2.waitKey(20) & 0xFF == ord('q'):
            break

        frame_count += 1
    cap.release()
    cv2.destroyAllWindows()
# Example usage
# Load a lighter YOLO model (e.g., YOLOv8-tiny) and set the video path


In [ ]:
Video_Path_from_youtube = r"https://www.youtube.com/watch?v=jMVYZ9uMkwI"
test = process_live_video(Video_Path_from_youtube, frame_skip=3, ConfidenceThreshold = 0.25 ,ShowVideo = True)

# Extract and data result from YOLO8 from

In [1]:
import cv2
import pandas as pd
from ultralytics import YOLO
from IPython.display import clear_output
from datetime import datetime, timedelta

# Function to process video with optimizations
def process_video_GetResults(video_path, frame_skip=1, ConfidenceThreshold = 0.75, ShowVideo = False, Resolution = (0, 0), excel_file='predictions.xlsx'):
    
    # Load video
    model= YOLO('yolov8n.pt')
    cap = cv2.VideoCapture(video_path)

    all_detections = []  # To store all detections

    
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        #clear_output(wait=True)
        if not ret:
            break
         # Skip frames if needed
        if frame_count % frame_skip != 0:
             frame_count += 1
             continue

        # Optionally resize the frame for faster processing
        if Resolution != (0,0):
            frame = cv2.resize(frame, Resolution)   
        current_time = datetime.now()    
        
        # Use the model to track objects in the frame
        results = model.track(frame, show=False, persist=True, conf=ConfidenceThreshold)
        print('=================================================================')

        detections = extract_detections(results, current_time, frame_count, model.names)
        all_detections.extend(detections)  # Add detections to the main list
       
        print('=================================================================')
        
        if ShowVideo:
            annotated_frame = results[0].plot()
            cv2.imshow("FRAME",annotated_frame)
            del annotated_frame

        del frame 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()
    
    df = pd.DataFrame(all_detections)

    # Append to an existing Excel file or create a new one
    with pd.ExcelWriter(excel_file, mode='a', if_sheet_exists='overlay', engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='Detections')
# Example usage
# Load a lighter YOLO model (e.g., YOLOv8-tiny) and set the video path



In [3]:
def extract_detections(results,timestamp, frame_number, class_names):
    detections = []

    # Loop over each result to extract details
    for r in results:
        for det in r.boxes:
            # Get class index and map it to class name
            class_index = int(det.cls[0])
            class_name = class_names[class_index]
            
            detection = {
                'Time': timestamp.strftime('%Y-%m-%d %H:%M:%S'),  # Format time as string
                'FrameNumber': frame_number,   # Use frame number as timestamp; can be modified to use actual time
                'Object': class_name,  # Class name (e.g., person, car)
                'ID': int(det.id[0]) if det.id is not None else None,  # Unique ID for tracking
                'Confidence': float(det.conf[0])  # Confidence score
            }
            detections.append(detection)

    return detections

In [ ]:
Video_Path = r"C:\Users\KeyhanAzarjoo\Python Training Code\Python-Yolo8-object-Detection\video360p.mp4"
Save_Path = 0
test = process_video_GetResults(Video_Path, frame_skip=60, ConfidenceThreshold = 0.50, ShowVideo = True, Resolution = (0, 0), excel_file = Save_Path)

# write predicted results in a Textfile with CSV format Each some second

In [377]:
# Function to extract detected objects and their details
def extract_detections(results, timestamp, class_names):
    detections = []
    for r in results:
        for det in r.boxes:
            class_index = int(det.cls[0])
            class_name = class_names[class_index]
            detection = {
                'Time': timestamp.strftime('%Y-%m-%d %H:%M:%S'),
                'Object': class_name,
                'ID': int(det.id[0]) if det.id is not None else None,
                'Confidence': float(det.conf[0])
            }
            detections.append(detection)
    return detections

In [379]:
# Function to append data to a CSV file
def append_to_csv(detections, csv_file):
    df = pd.DataFrame(detections)
    if not os.path.exists(csv_file):
        df.to_csv(csv_file, index=False, mode='w')
    else:
        df.to_csv(csv_file, index=False, mode='a', header=False)

In [ ]:
import cv2
import torch
import pandas as pd
from ultralytics import YOLO
from datetime import datetime, timedelta
import os


# Main function to process video and save results to CSV
def process_video_GetResults(video_path, frame_skip=1, ConfidenceThreshold=0.75, ShowVideo=False, Resolution=(0, 0), csv_file='predictions.csv', save_interval=1):
    model = YOLO('yolov8n.pt')
    cap = cv2.VideoCapture(video_path)
    frame_rate = cap.get(cv2.CAP_PROP_FPS)
    start_time = datetime.now()

    frame_count = 0
    all_detections = []
    last_save_time = start_time

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_skip != 0:
            frame_count += 1
            continue

        if Resolution != (0, 0):
            frame = cv2.resize(frame, Resolution)

        timestamp = start_time + timedelta(seconds=frame_count / frame_rate)
        results = model.track(frame, show=False, persist=True, conf=ConfidenceThreshold)
        detections = extract_detections(results, timestamp, model.names)
        all_detections.extend(detections)

        current_time = datetime.now()
        if (current_time - last_save_time).total_seconds() >= save_interval:
            append_to_csv(all_detections, csv_file)
            last_save_time = current_time
            all_detections = []

        if ShowVideo:
            annotated_frame = results[0].plot()
            cv2.imshow("FRAME", annotated_frame)
            del annotated_frame

        del frame
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

    if all_detections:
        append_to_csv(all_detections, csv_file)

# Example usage
Video_Path = r"C:\Users\KeyhanAzarjoo\Python Training Code\Python-Yolo8-object-Detection\video360p.mp4"
Save_Path = r"C:\Users\KeyhanAzarjoo\Python Training Code\Python-Yolo8-object-Detection\predictions.csv"

process_video_GetResults(Video_Path, frame_skip=30, ConfidenceThreshold=0.50, ShowVideo=True, Resolution=(0, 0), csv_file=Save_Path, save_interval=1)


# Extract light Version (NCNN) From YOLO v8

In [8]:
# Create a light version of ncnn from YOLO
# Export to NCNN: Converting Your YOLOv8 Model

In [13]:
#You can expand model compatibility and deployment flexibility by converting YOLOv8 models to NCNN format.

# ncnn is a light version of your model to run on embadded system or phone or websites.
#Embedded Systems and IoT Devices: If you find that running inference on a Raspberry Pi with the Ultralytics Guide isn't fast enough, switching to an NCNN exported model could help speed things up. NCNN is great for devices like Raspberry Pi and NVIDIA Jetson, especially in situations where you need quick processing right on the device.

#Desktop and Server Deployment: Capable of being deployed in desktop and server environments across Linux, Windows, and macOS, supporting development, training, and evaluation with higher computational capacities.

#reference : https://docs.ultralytics.com/integrations/ncnn/#deployment-options-with-ncnn

In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.pt")

# Export the model to NCNN format
model.export(format="ncnn")  # creates '/yolov8n_ncnn_model'

# Load the exported NCNN model
ncnn_model = YOLO("./yolov8n_ncnn_model")